# 📎 Embedding Decathlon HTML Docs into Pinecone using Langchain

In [24]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from dotenv import load_dotenv
import os

Initializing Environment Variables

In [36]:
load_dotenv(override=True)

True

In [37]:
OPENAI_API_KEY = os.getenv("openai_api_key")
PINECONE_API_KEY = os.getenv("pinecone_api_key")
PINECONE_ENV_KEY = os.getenv("pinecone_env_key")
PINECONE_INDEX = os.getenv("pinecone_index")

Loading Decathlon Documents from the Decathlon website (Home, FAQ, Warranty, Product Returns etc...)

In [10]:
loader = WebBaseLoader(["https://www.decathlon.ma/", "https://www.decathlon-united.com/fr", "https://www.decathlon.ma/content/63-garantie", "https://www.decathlon.ma/content/87-retour-echange", 
                        "https://www.decathlon.ma/content/88-cartecadeaux", "https://www.decathlon.ma/content/96-cliquez-et-retirez", "https://www.decathlon.ma/content/85-echo-conception",
                        "https://www.decathlon.ma/content/86-nos-innovations", "https://www.decathlon.ma/module/decab2b/b2b?icn=HomePage-Footer-DecathlonPRO", "https://www.decathlon.ma/page/acheter-en-ligne.html", 
                        "https://www.decathlon.ma/page/consulter-stock.html", "https://www.decathlon.ma/content/1-livraison", "https://www.decathlon.ma/page/rappelproduit.html", "https://www.decathlon.ma/page/cgu_cgv.html"
                        "https://www.decathlon.ma/page/donnees-personnelles-et-cookies.html", "https://www.decathlon.ma/page/conditions-de-publication-des-avis.html", "https://www.decathlon.ma/page/mention_legale.html",
                        "https://www.decathlon.ma/content/102-decathlon-occasion?icn=ServicesPage-occasion", "https://www.decathlon.ma/5080-promotions?icn=HomePage-Menu-Promotions", "https://www.decathlon.ma/nous-contacter"])
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(docs)

In [11]:
len(docs)

1547

We get 1547 Documents after loading the HTML Files using Langchain WebBaseLoader and then splittingn them

Now we need to embedd the documents and then store them into a Pincecone Index

In [12]:
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [28]:
from langchain.vectorstores import Pinecone
import pinecone

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV_KEY
)

In [39]:
index_name=PINECONE_INDEX

In [42]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

Storing the docs into the Pinecone index.

In [43]:
doc_store = Pinecone.from_texts([d.page_content for d in docs], embedding, index_name=index_name)

## ⛓️ Langchain LLM Chatbot

Now let's use gpt-3.5 OpenAI model to create a chatbot that can answer customers questions

In [19]:
def get_response_from_query(db, query, k=4):
        """
        Function that generates a response to a customer question using the gpt-3.5-model and the docs provided
        """

        docs = db.similarity_search(query, k=k)
        docs_page_content = " ".join([d.page_content for d in docs])

        chat = ChatOpenAI(model_name="gpt-3.5-turbo")

        # Template to use for the system message prompt
        template = """
            Tu es un assistant utile qui peut répondre aux questions des clients sur une plateforme de commerce électronique qui vend des equipements de sport
            nommé "Decathlon" basé sur les données html trouvées sur leur site Web : {docs}
            
            D'abord tu classe le sentiment de la question ou de la phrase du client, et tu utilisez uniquement les informations factuelles du code HTML pour répondre à la question, 
            et tu réponds en prenant compte du sentiment du client.
            
            Si tu sents que tu n'as pas assez d'informations pour répondre à la question, réponds par "Désolé, je ne sais pas la réponse à ta question.".
            
            Tes réponses doivent être détaillées.
            """

        system_message_prompt = SystemMessagePromptTemplate.from_template(template)

        # Human question prompt
        human_template = "Réponds à la question suivante: {question}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        chat_prompt = ChatPromptTemplate.from_messages(
            [system_message_prompt, human_message_prompt]
        )

        chain = LLMChain(llm=chat, prompt=chat_prompt)

        try:
            response = chain.run(question=query, docs=docs_page_content)
            response = response.replace("\n", "")
            return response
        except:
            return None

In [50]:
response = get_response_from_query(doc_store, "COMMENT EFFECTUER MON RETOUR PRODUIT EN MAGASIN ?")
response

"Pour effectuer un retour produit en magasin chez Decathlon, vous devez vous rendre dans l'un des magasins Decathlon au Maroc avec le produit que vous souhaitez retourner. Une fois sur place, vous pouvez soit échanger votre article, soit vous faire rembourser. Cette politique de retour est valable pour tous les produits achetés sur le site web de Decathlon au Maroc et dans l'ensemble de leurs magasins dans le pays. Cependant, pour les achats effectués à l'étranger, le retour est limité aux produits de marques passion Decathlon."

Using the already created Index to query the db and generate a response

In [51]:
docsearch = Pinecone.from_existing_index(index_name, embedding)

In [52]:
response = get_response_from_query(docsearch, "COMMENT EFFECTUER UN RETOUR DE PRODUIT EN MAGASIN ?")
response

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


"Pour effectuer un retour produit en magasin chez Decathlon Maroc, voici les étapes à suivre :1. Rendez-vous dans n'importe quel magasin Decathlon Maroc avec votre produit et votre preuve d'achat.2. Présentez votre produit et votre preuve d'achat à un membre du personnel du magasin.3. Vous pouvez choisir entre un échange ou un remboursement.4. Si vous échangez votre produit, vous pouvez choisir un produit similaire de même valeur ou payer la différence si le nouveau produit est plus cher.5. Si vous demandez un remboursement, vous serez remboursé(e) sur votre moyen de paiement initial.Notez que cette politique de retour est valable pour tous les produits achetés sur www.decathlon.ma et dans l'ensemble des magasins Decathlon Marocains. Pour les achats effectués à l’étranger, le retour est limité aux produits de marques passion DECATHLON."